In [ ]:
from tkinter import * 
from tkinter import messagebox 

def tell_weather() : 

	import requests, json 
	api_key = "0eb682772b5416a6bc4c8bfadab5de7c" 
	base_url = "http://api.openweathermap.org/data/2.5/forecast?id=524901&APPID=0eb682772b5416a6bc4c8bfadab5de7c" 
	city_name = city_field.get() 
	complete_url = base_url + "appid =" + api_key + "&q =" + city_name 
	response = requests.get(complete_url) 
	x = response.json()  
 
	if x["cod"] != "404" : 
		y = x["main"] 
		current_temperature = y["temp"] 
		current_pressure = y["pressure"] 
		current_humidity = y["humidity"] 
		z = x["weather"] 
		weather_description = z[0]["Description"]  
		temp_field.insert(15, str(current_temperature) + " Kelvin") 
		atm_field.insert(10, str(current_pressure) + " Pa") 
		humid_field.insert(15, str(current_humidity) + " %") 
		desc_field.insert(10, str(weather_description) + "© Kuber Khanna" ) 			
		 
	else : 
		messagebox.showerror("Error", "City Not Found \n \n"
							"Please enter valid city name !") 
		city_field.delete(0, END)  
	

def clear_all() : 
	city_field.delete(0, END) 
	temp_field.delete(0, END) 
	atm_field.delete(0, END) 
	humid_field.delete(0, END) 
	desc_field.delete(0, END) 
	city_field.focus_set() 
 
if __name__ == "__main__" : 
	root = Tk() 
	root.title("Weather API") 
	root.configure(background = "light green") 
	root.geometry("425x175") 
 
	headlabel = Label(root, text = "Weather API", 
				fg = 'black', bg = 'red') 
	label1 = Label(root, text = "City : ", 
				fg = 'black', bg = 'dark green') 
	label2 = Label(root, text = "Temperature :", 
				fg = 'black', bg = 'dark green') 
	label3 = Label(root, text = "Atmospheric Pressure :", 
				fg = 'black', bg = 'dark green') 
	label4 = Label(root, text = "Humidity :", 
				fg = 'black', bg = 'dark green') 
	label5 = Label(root, text = "Description :", 
				fg = 'black', bg = 'dark green') 
 
	headlabel.grid(row = 0, column = 1, sticky ="E") 
	label1.grid(row = 1, column = 0, sticky ="E") 
	label2.grid(row = 3, column = 0, sticky ="E") 
	label3.grid(row = 4, column = 0, sticky ="E") 
	label4.grid(row = 5, column = 0, sticky ="E") 
	label5.grid(row = 6, column = 0, sticky ="E") 
 
	city_field = Entry(root) 
	temp_field = Entry(root) 
	atm_field = Entry(root) 
	humid_field = Entry(root) 
	desc_field = Entry(root) 
 
	city_field.grid(row = 1, column = 1, ipadx ="100") 
	temp_field.grid(row = 3, column = 1, ipadx ="100") 
	atm_field.grid(row = 4, column = 1, ipadx ="100") 
	humid_field.grid(row = 5, column = 1, ipadx ="100") 
	desc_field.grid(row = 6, column = 1, ipadx ="100") 
 
	button1 = Button(root, text = "Submit", bg = "red", 
					fg = "black", command = tell_weather)  
	button2 = Button(root, text = "Clear", bg = "red", 
					fg = "black", command = clear_all)  
	button1.grid(row = 2, column = 1) 
	button2.grid(row = 7, column = 1) 
 
 #Run in loop
	root.mainloop() 

In [ ]:
import tkinter as tk
import pyowm
import time
import json
from datetime import datetime
from collections import OrderedDict #######-------#######=======#######~~~~~~~#######_______#######


class WeatherInfo(tk.Tk):
    templates = OrderedDict([
        ('temp', 'Temperature: {temp:.1f} C'),
        ('humid', 'Humidity: {humid}%'),
        ('status', 'Status: {status}'),
        ('sunrise', 'Sunrise at {sunrise:%H:%M:%S}'),
        ('sunset', 'Sunset at {sunset:%H:%M:%S}'),
        ('day_length', 'Day length: {day_length:.1f} h'),
        ('night_length', 'Night length: {night_length:.1f} h')])

    def __init__(self):
        tk.Tk.__init__(self)
        self.wm_title('Weather API')
        self.tk_info = {key: tk.StringVar(
            self, value='') for key in WeatherInfo.templates}
#City finding
        self.ask = tk.LabelFrame(self, text='Location')
        self.ask.pack(fill='both', expand='yes')
        self.kw_label = tk.Label(self.ask, text='City:')
        self.kw_label.pack(side=tk.LEFT)
        self.kw = tk.Entry(self.ask)
        self.kw.pack(side=tk.LEFT)
        self.rb = tk.Button(self.ask, text='BROWSE', command=self.main)
        self.rb.pack(side=tk.RIGHT)
        self.owm = pyowm.OWM('*CENSORED*')

        self.output = tk.LabelFrame(self, text='Information')
        self.output.pack(fill='both', expand='yes')
        self.labels = []
        for key in WeatherInfo.templates:
            self.labels.append(
                tk.Label(self.output, textvariable=self.tk_info[key]).pack())
        button = tk.Button(master=self, text='Quit', command=self._quit)
        button.pack(side=tk.BOTTOM)

    def search(self):
        obs = self.owm.weather_at_place(self.kw.get())
        try:
            return json.loads(obs.get_weather().to_JSON())
        except AttributeError:
            self.tk_info['temp'].set('CITY')

    def parse(self, w):
        parsed_weather = {'temp': w['temperature']['temp'] - 273.15,
                          'humid': w['humidity'],
                          'status': w['status'],
                          'sunrise': datetime.fromtimestamp(w['sunrise_time']),
                          'sunset': datetime.fromtimestamp(w['sunset_time']),
                          'day_length': (w['sunset_time'] - w['sunrise_time']) / 3600,
                          'night_length': 24 - (w['sunset_time'] - w['sunrise_time']) / 3600}
        return parsed_weather

    def update(self, report):
        for key, template in WeatherInfo.templates.items():
            self.tk_info[key].set(template.format(**report))
#Update of the search
    def main(self):
        report = self.search()
        if report:
            self.update(self.parse(report))
#Ending the program
    def _quit(self):
        self.quit()
        self.destroy()
#Main Loop
if __name__ == "__main__":
    app = WeatherInfo()
    app.mainloop()